<a href="https://colab.research.google.com/github/sagarprince/expense_income_train_model/blob/main/ExpenseIncomeModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
pip install tensorflow pandas scikit-learn

In [55]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
df = pd.read_csv("transaction_data.csv")

# Separate the features and labels
transaction_type = df["Transaction_Type"]
description = df["Description"]
category = df["Category"]
label = df["Label"]

# Encode labels
label_encoder_category = LabelEncoder()
label_encoder_label = LabelEncoder()
category = label_encoder_category.fit_transform(category)
label = label_encoder_label.fit_transform(label)

# Split the dataset into train and test sets
transaction_type_train, transaction_type_test, description_train, description_test, category_train, category_test, label_train, label_test = train_test_split(
    transaction_type, description, category, label, test_size=0.2, random_state=42
)

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(description_train)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(description_train)
test_sequences = tokenizer.texts_to_sequences(description_test)

# Pad sequences for equal length
max_seq_length = 100
train_data_pad = pad_sequences(train_sequences, maxlen=max_seq_length)
test_data_pad = pad_sequences(test_sequences, maxlen=max_seq_length)

# Define the model
input_description = Input(shape=(max_seq_length,))
embedding_description = Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_seq_length)(input_description)
lstm_description = LSTM(128)(embedding_description)

input_transaction_type = Input(shape=(1,))
embedding_transaction_type = Embedding(len(set(transaction_type)), 8)(input_transaction_type)
flatten_transaction_type = tf.keras.layers.Flatten()(embedding_transaction_type)

concatenated_features = Concatenate()([lstm_description, flatten_transaction_type])
dense1 = Dense(64, activation="relu")(concatenated_features)
dense2 = Dense(32, activation="relu")(dense1)
output_category = Dense(len(set(category)), activation="softmax", name="category")(dense2)
output_label = Dense(len(set(label)), activation="softmax", name="label")(dense2)

model = Model(inputs=[input_description, input_transaction_type], outputs=[output_category, output_label])

# Compile the model
model.compile(
    loss={"category": "sparse_categorical_crossentropy", "label": "sparse_categorical_crossentropy"},
    optimizer="adam",
    metrics=["accuracy"]
)

# Encode transaction type
label_encoder_transaction_type = LabelEncoder()
transaction_type_train_encoded = label_encoder_transaction_type.fit_transform(transaction_type_train)
transaction_type_test_encoded = label_encoder_transaction_type.transform(transaction_type_test)

# Train the model
model.fit(
    [train_data_pad, transaction_type_train_encoded],
    [category_train, label_train],
    epochs=1000,
    batch_size=64,
    validation_data=(
        [test_data_pad, transaction_type_test_encoded],
        [category_test, label_test]
    )
)

# Evaluate the model
loss, category_loss, label_loss, category_accuracy, label_accuracy = model.evaluate(
    [test_data_pad, transaction_type_test_encoded],
    [category_test, label_test]
)
print("Test Loss:", loss)
print("Category Loss:", category_loss)
print("Label Loss:", label_loss)
print("Category Accuracy:", category_accuracy)
print("Label Accuracy:", label_accuracy)

# Save the model
model.save("transaction_model.h5")
print("Trained Model Saved")


Epoch 1/1000
1/1 [==============================] - 5s 5s/step - loss: 2.7781 - category_loss: 2.0800 - label_loss: 0.6982 - category_accuracy: 0.0952 - label_accuracy: 0.2381 - val_loss: 2.7608 - val_category_loss: 2.0762 - val_label_loss: 0.6847 - val_category_accuracy: 0.0000e+00 - val_label_accuracy: 0.6667
Epoch 2/1000
1/1 [==============================] - 0s 188ms/step - loss: 2.7405 - category_loss: 2.0731 - label_loss: 0.6674 - category_accuracy: 0.1429 - label_accuracy: 0.9524 - val_loss: 2.7472 - val_category_loss: 2.0769 - val_label_loss: 0.6704 - val_category_accuracy: 0.0000e+00 - val_label_accuracy: 0.6667
Epoch 3/1000
1/1 [==============================] - 0s 201ms/step - loss: 2.7046 - category_loss: 2.0663 - label_loss: 0.6382 - category_accuracy: 0.1429 - label_accuracy: 0.9524 - val_loss: 2.7365 - val_category_loss: 2.0794 - val_label_loss: 0.6571 - val_category_accuracy: 0.0000e+00 - val_label_accuracy: 0.6667
Epoch 4/1000
1/1 [==============================] - 0s 

In [61]:
# Load the model
loaded_model = tf.keras.models.load_model("transaction_model.h5")
print("Model Loaded");

# Predict on new data
new_data = "INR 9602.00 debited to HDFC Bank A/C No XXXXXXXXXX3078 towards BAJAJ FINANCE LTD / 25038107  with UMRN HDFC7020910220007476"
# new_data = "HDFC Bank: Rs 850.00 debited from a/c **3078 on 28-02-23 to VPA paytmqr2810050501011ivc7tjxi39r@paytm(UPI Ref No 305969683136). Not you? Call on 18002586161 to report"
# new_data = "You've withdrawn Rs.1000 On HDFC Bank Debit Card xx0229 At YBL LAXMIPURI On 2023-07-01:17:20:47 Avl bal: Rs.180761.6 Not you?Call 18002586161"
# new_data = "Alert You've spent Rs 3478.00 On HDFC Bank Debit Card xx229 at Spicey on 2023-07-16 Avl Bal 232323"
new_data_sequence = tokenizer.texts_to_sequences([new_data])
new_data_pad = pad_sequences(new_data_sequence, maxlen=max_seq_length)

# Iterate over transaction types to find the correct position
for i, tt in enumerate(transaction_type):
    tt_encoded = label_encoder_transaction_type.transform([tt])
    predictions_category, predictions_label = loaded_model.predict([new_data_pad, tt_encoded])
    predicted_category = label_encoder_category.inverse_transform([predictions_category.argmax()])[0]
    predicted_label = label_encoder_label.inverse_transform([predictions_label.argmax()])[0]

    if predicted_category != "unknown" and predicted_label != "unknown":
        break

print("Predicted Category:", predicted_category)
print("Predicted Label:", predicted_label)

Model Loaded
1/1 [==============================] - 1s 769ms/step
Predicted Category: Health
Predicted Label: Expense
